In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats
import statistics
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Представление данных

In [160]:
df = pd.read_csv(r'C:\Users\pavlu\PycharmProjects\A-Statistical-Analysis-ML-workflow-of-Titanic\data\train.csv')

In [161]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [162]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Классифицируем признаки: Survived - категориальный; Pclass - порядковый (как количество звездочек у отеля); Name - категориальный; Sex - категориальный; Age - количественный; SibSp - Siblings/Spouses Aboard - количественный; Parch - # of parents / children aboard - количественный; Ticket - категориальный; Fare - количественный; Cabin - категориальный; Embarked - Port of Embarkation - категориальный.

### Поиск и обработка пропусков и дубликатов

In [163]:
print(f'Стоблцы с пропусками: {df.columns[df.isnull().any()].tolist()}')
df.isnull().sum()

Стоблцы с пропусками: ['Age', 'Cabin', 'Embarked']


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Первый способ работы с пропусками - заменить значение средним. Достоинство данного метода - простота и возможность сохранить данные. Как правило, подстановка средних значений не принимает во внимание корреляцию между признаками. Кроме того, такая подстановка уменьшает дисперсию данных и повышает уровень ошибок. Это приводит к менее точной модели и более узким доверительным интервалам. Данный вид заполнения пропусков можно применить к столбцу Age.

In [164]:
df["Age"].fillna(df["Age"].mean(),inplace= True)

Еще один способ работы с пропусками - удаление строк с nans. Этот способ плох тем, что мы уменьшаем количество данных, но зато не привносим в них шума. Этот способ можно применить для Embarked, т.к. отсутствуют всего два экземпляра.

In [165]:
df.dropna(subset = ["Embarked"], inplace=True)

Номер кабины может быть идентификатором личности, поэтому заменять случайными/выборочными значениями его нельзя. Этот признак однозначно не является целевым, поэтому его можно пока что просто не учитывать.

In [166]:
df.isnull().sum() # пропуски для Embarked и Age убраны

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [167]:
df[df.duplicated()] # дубликатов нет

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


### Анализ данных

In [168]:
fig = px.box(df, x='Survived', y="Fare",color= 'Pclass')
fig.show()

In [169]:
fig = px.box(df, x='Survived', y="Fare")
fig.show()

In [170]:
fig = px.box(df, x='Survived', y='Age',color= 'Pclass')
fig.show()

In [171]:
fig = px.box(df, x='Survived', y='Age')
fig.show()

In [172]:
fig = px.box(df, x='Survived', y='Age',color= 'Sex')
fig.show()

Заметны выбросы по возрасту и цене билета, так как эти признаки кажутся важными для определения выживания, то выбросы стоит модифицировать в отдельные категории относительно boxplot - группа "нижних" и "верхних" выбросов. Далее найдем для каждой группы медиану и подставим ее вместо значения.

In [173]:
def outliers_iqr(x):
    quartile_1, quartile_3 = np.percentile(x, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where(x < lower_bound), np.where((x > upper_bound))
# для возраста
bottom_outliers_ix, top_outliers_ix = outliers_iqr(df.Age)
bottom_age_median = statistics.median(df.Age.iloc[bottom_outliers_ix])
top_age_median = statistics.median(df.Age.iloc[top_outliers_ix])
df.Age.iloc[bottom_outliers_ix] = bottom_age_median
df.Age.iloc[top_outliers_ix] = top_age_median
# для цены билета
fare_greater_200_ix = np.where((df.Fare > 200))
big_fare_median = statistics.median(df.Fare.iloc[fare_greater_200_ix])
df.Fare.iloc[fare_greater_200_ix] = big_fare_median

C:\Users\pavlu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Кто выжил?

In [174]:
n_of_men = df[df['Sex'] == 'male'].shape[0]
n_of_women = df[df['Sex'] == 'female'].shape[0]
n_of_1_class = df[df['Pclass'] == 1].shape[0]
n_of_2_class = df[df['Pclass'] == 2].shape[0]
n_of_3_class = df[df['Pclass'] == 3].shape[0]
min_age = int(df.Age.min())
max_age = int(df.Age.max())
mean_age = round(df.Age.mean(), 3)
survived = df[df['Survived'] == 1].shape[0]
unsurvived = df[df['Survived'] == 0].shape[0]

print(f"На Титанике было {n_of_men} мужчин и {n_of_women} женщин. \n" 
      f"{n_of_1_class} пассажиров являлись пассажирами 1ого класса, {n_of_2_class} - 2ого класса и {n_of_3_class} - 3-его класса. \n"
      f"Средний возраст пассажиров - {mean_age}, примерный диапозон: {min_age} - {max_age} лет. \n"
      f"Статистика выживаемости следующая: {survived} пассажиров выжило, {unsurvived}, к сожалению, нет.")

На Титанике было 577 мужчин и 312 женщин. 
214 пассажиров являлись пассажирами 1ого класса, 184 - 2ого класса и 491 - 3-его класса. 
Средний возраст пассажиров - 29.598, примерный диапозон: 1 - 61 лет. 
Статистика выживаемости следующая: 340 пассажиров выжило, 549, к сожалению, нет.


In [175]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Age", "Fare", "Pclass", "Sex"))

fig.add_trace(go.Histogram(x=df.Age),row=1, col=1)

fig.add_trace(go.Histogram(x=df.Fare),row=1, col=2)

fig.add_trace(go.Histogram(x=df.Pclass),row=2, col=1)

fig.add_trace(go.Histogram(x=df.Sex),row=2, col=2)

fig.update_layout(height=800, width=900,
                  title_text="Гистограммы основных фич")

fig.show()

In [176]:
def gender_live(df):
    df_gender_surv = df.groupby(['Survived', 'Sex']).size()
    df_gender_surv = df_gender_surv.reset_index(name='count')
    male_survived = df_gender_surv.loc[(df_gender_surv['Survived'] == 1) & (df_gender_surv['Sex'] == 'male')]['count'].values[0]
    female_survived = df_gender_surv.loc[(df_gender_surv['Survived'] == 1) & (df_gender_surv['Sex'] == 'female')]['count'].values[0]
    n_men = sum(df_gender_surv.loc[df_gender_surv['Sex'] == 'male']['count'])
    n_women = sum(df_gender_surv.loc[df_gender_surv['Sex'] == 'female']['count'])
    percent_men_surv = male_survived / n_men
    percent_women_surv = female_survived / n_women
    return percent_men_surv, percent_women_surv

def wealth_live(df):
    df_pclass_surv = df.groupby(['Survived', 'Pclass']).size()
    df_pclass_surv = df_pclass_surv.reset_index(name='count')
    pclass1_survived = df_pclass_surv.loc[(df_pclass_surv['Survived'] == 1) & (df_pclass_surv['Pclass'] == 1)]['count'].values[0]
    pclass23_survived = df_pclass_surv.loc[(df_pclass_surv['Survived'] == 1) & ((df_pclass_surv['Pclass'] == 2) | (df_pclass_surv['Pclass'] == 3))]['count'].values[0]
    n_pclass1 = sum(df_pclass_surv.loc[df_pclass_surv['Pclass'] == 1]['count'])
    n_pclass23 = sum(df_pclass_surv.loc[((df_pclass_surv['Pclass'] == 2) | (df_pclass_surv['Pclass'] == 3))]['count'])
    percent_pclass1_surv = pclass1_survived / n_pclass1
    percent_pclass23_surv = pclass23_survived / n_pclass23
    return percent_pclass1_surv, percent_pclass23_surv

def fare_live(df):
    df.loc[df.Fare <= 15, 'Fare_Live'] = 0
    df.loc[(15 < df.Fare) & (df.Fare < 45), 'Fare_Live'] = 1
    df.loc[df.Fare >= 100, 'Fare_Live'] = 2
    df_age_surv = df.groupby(['Survived', 'Fare_Live']).size()
    df_age_surv = df_age_surv.reset_index(name='count')
    poor_survived = df_age_surv.loc[(df_age_surv['Survived']==1) & (df_age_surv['Fare_Live'] == 0)]['count'].values[0]
    middle_survived = df_age_surv.loc[(df_age_surv['Survived']==1) & (df_age_surv['Fare_Live'] == 1)]['count'].values[0]
    rich_survived = df_age_surv.loc[(df_age_surv['Survived']==1) & (df_age_surv['Fare_Live'] == 2)]['count'].values[0]
    n_poor = sum(df_age_surv.loc[df_age_surv['Fare_Live'] == 0]['count'])
    n_middle = sum(df_age_surv.loc[df_age_surv['Fare_Live'] == 1]['count'])
    n_rich = sum(df_age_surv.loc[df_age_surv['Fare_Live'] == 2]['count'])
    percent_poor_surv = poor_survived / n_poor
    percent_middle_surv = middle_survived / n_middle
    percent_rich_surv = rich_survived / n_rich
    return percent_poor_surv, percent_middle_surv, percent_rich_surv

def age_live(df):
    df.loc[df.Age < 25, 'Age_Live'] = 0
    df.loc[(25 <= df.Age) & (df.Age <= 50), 'Age_Live'] = 1
    df.loc[df.Age > 50, 'Age_Live'] = 2
    df_age_surv = df.groupby(['Survived', 'Age_Live']).size()
    df_age_surv = df_age_surv.reset_index(name='count')
    young_survived = df_age_surv.loc[(df_age_surv['Survived']==1) & (df_age_surv['Age_Live'] == 0)]['count'].values[0]
    middle_survived = df_age_surv.loc[(df_age_surv['Survived']==1) & (df_age_surv['Age_Live'] == 1)]['count'].values[0]
    ageing_survived = df_age_surv.loc[(df_age_surv['Survived']==1) & (df_age_surv['Age_Live'] == 2)]['count'].values[0]
    n_young = sum(df_age_surv.loc[df_age_surv['Age_Live'] == 0]['count'])
    n_middle = sum(df_age_surv.loc[df_age_surv['Age_Live'] == 1]['count'])
    n_ageing = sum(df_age_surv.loc[df_age_surv['Age_Live'] == 2]['count'])
    percent_young_surv = young_survived / n_young
    percent_middle_surv = middle_survived / n_middle
    percent_ageing_surv = ageing_survived / n_ageing
    return percent_young_surv, percent_middle_surv, percent_ageing_surv

In [177]:
percent_men_surv, percent_women_surv = gender_live(df)
percent_pclass1_surv, percent_pclass23_surv = wealth_live(df)
percent_poor_surv, percent_middle_surv, percent_rich_surv = fare_live(df)
percent_young_surv, percent_middle_surv, percent_ageing_surv = age_live(df)

In [194]:
fig = make_subplots(
    rows=2, cols=2, specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]],
    subplot_titles=("Sex", "Pclass", "Fare", "Age"))

fig.add_trace(go.Pie(labels=['men', 'women'], values=list((percent_men_surv, percent_women_surv))), row=1, col=1)

fig.add_trace(go.Pie(labels=['pclass1', 'pclass23'], values=list((percent_pclass1_surv, percent_pclass23_surv))),row=1, col=2)

fig.add_trace(go.Pie(labels=['poor', 'middle', 'rich'], values=list((percent_poor_surv, percent_middle_surv, percent_rich_surv))),row=2, col=1)

fig.add_trace(go.Pie(labels=['young', 'middle_age', 'ageing'], values=list((percent_young_surv, percent_middle_surv, percent_ageing_surv))),row=2, col=2)

fig.update_layout(height=700, width=900,
                  title_text="Распределение выживших")

fig.show()